## Group3
### Members
1.Andrea Ulloa 
2.Ana Angulo (20171627)
3.Angela Coapaza (20171636) 

## An Inferencial Problem: The College-Educated Wage Gap

Using the data from the March Supplement of the U.S. Current Population Survey (2015) , in  this lab, we focus on the following inference question:

What is the difference in predicted wages between men and women with the same job-relevant characteristics? However, we only focus on the subset of college-educated workers.

To investigate the gender wage gap in this subset, we consider the following log-linear regression model

\begin{align}
\log(Y) &= \beta'X + \epsilon\\
&= \beta_1 D  + \beta_2' W + \epsilon,
\end{align}

where $D$ is the indicator of being female ($1$ if female and $0$ otherwise) and the
$W$'s are controls explaining variation in wages*. Considering transformed wages by the logarithm, we are analyzing the relative difference in the payment of men and women.

*The variables used to restrict the database (scl and clg) are no longer part of the W's controls anymore.

## 1. Data Analysis

We consider the same subsample of the U.S. Current Population Survey (2015). Let us load the data set.

In [56]:
using Pkg

Pkg.add("DataFrames")
Pkg.add("Dates")
Pkg.add("CategoricalArrays")
Pkg.add("RData")
Pkg.add("Plots")

using RData
using DataFrames
using Dates
using Statistics  #upload data of R format 
using CategoricalArrays # categorical data
using Plots

   Resolving package versions...
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Manifest.toml`


In [5]:
#Pkg.build("GR")
#Pkg.build("Plots")

    Building GR → `C:\Users\ANA\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\af237c08bda486b74318c8070adb96efa6952530\build.log`
    Building GR → `C:\Users\ANA\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\af237c08bda486b74318c8070adb96efa6952530\build.log`


In [55]:
rdata_read = load("../../data/wage2015_subsample_inference.RData")
data = rdata_read["data"]
names(data)
println("Number of Rows : ", size(data)[1],"\n","Number of Columns : ", size(data)[2],) #rows and columns

Number of Rows : 5150
Number of Columns : 20


***Variable description***

- occ : occupational classification
- ind : industry classification
- lwage : log hourly wage
- sex : gender (1 female) (0 male)
- shs : some high school
- hsg : High school graduated
- scl : Some College
- clg: College Graduate
- ad: Advanced Degree
- ne: Northeast
- mw: Midwest
- so: South
- we: West
- exp1: experience

As we mention before, we focus only on a subset of the data. For this porpouse, we use scl and clg variables.

In [75]:
#subset(data, :scl =>ByRow(==(1)), :clg =>ByRow(==(1)))
#data1= data[(data[:,"scl"].==1) .| (data[:,"clg"].==1),:] 
data = filter(row -> (row.scl == 1 || row.clg == 1) , data) 
data.occ2

3068-element CategoricalArray{String,1,UInt8}:
 "11"
 "10"
 "6"
 "5"
 "10"
 "11"
 "11"
 "1"
 "20"
 "15"
 "11"
 "19"
 "1"
 ⋮
 "4"
 "21"
 "2"
 "10"
 "17"
 "8"
 "1"
 "10"
 "8"
 "2"
 "16"
 "13"

To start our (causal) analysis, we compare the sample means given gender:

In [57]:
#Z = select(data, ["lwage","sex","shs","hsg","ad","ne","mw","so","we","exp1"])

#data_female = filter(row -> row.sex == 1, data)
#Z_female = select(data_female,["lwage","sex","shs","hsg","ad","ne","mw","so","we","exp1"] )

#data_male = filter(row -> row.sex == 0, data)
#Z_male = select(data_male,["lwage","sex","shs","hsg","ad","ne","mw","so","we","exp1"] )

Z = select(data, ["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"])

data_female = filter(row -> row.sex == 1, data)
Z_female = select(data_female,["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"] )

data_male = filter(row -> row.sex == 0, data)
Z_male = select(data_male,["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"] )
means = DataFrame( variables = names(Z), All = describe(Z, :mean)[!,2], Men = describe(Z_male,:mean)[!,2], Female = describe(Z_female,:mean)[!,2])


,variables,All,Men,Female
,String,Float64,Float64,Float64
1,lwage,3.00002,3.03841,2.9569
2,sex,0.470991,0.0,1.0
3,shs,0.0,0.0,0.0
4,hsg,0.0,0.0,0.0
5,scl,0.466754,0.481824,0.449827
6,clg,0.533246,0.518176,0.550173
7,ad,0.0,0.0,0.0
8,ne,0.226532,0.219347,0.234602
9,mw,0.265971,0.261245,0.27128


In particular, the table above shows that the difference in average logwage between men and women is equal to $0,0815$. Also, we can notice that other education dimensions are zero since we have restricted the database to people with only college education. 

In [58]:
mean(Z_female[:,:lwage]) - mean(Z_male[:,:lwage])

-0.08150855508735955

In [59]:
x=0.08150855508735955 -0.03834473367441493 
x

0.043163821412944614

Thus, the unconditional gender wage gap is about $8,2$% for the group of never married workers (women get paid less on average in our sample). In comparison to the general data base, the gender-gap is bigger by $4.32$%

This unconditional (predictive) effect of gender equals the coefficient $\beta$ in the univariate ols regression of $Y$ on $D$:

\begin{align}
\log(Y) &=\beta D + \epsilon.
\end{align}

We verify this by running an ols regression in Julia.

In [60]:
#install all the package that we can need
Pkg.add("Lathe")
Pkg.add("GLM") # package to run models 
Pkg.add("StatsPlots")
Pkg.add("MLBase")
Pkg.add("Tables")
Pkg.add("CovarianceMatrices") # robust standar error 
# Load the installed packages
using DataFrames
using Tables
using Lathe
using CSV
using GLM
using CovarianceMatrices

   Resolving package versions...
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\ANA\.julia\e

In [61]:
nocontrol_model = lm(@formula(lwage ~ sex), data)
nocontrol_est = GLM.coef(nocontrol_model)[2]
nocontrol_se = GLM.coeftable(nocontrol_model).cols[2][2]

nocontrol_se1 = stderror(HC1(), nocontrol_model)[2]
println("The estimated gender coefficient is ", nocontrol_est ," and the corresponding robust standard error is " ,nocontrol_se1)

The estimated gender coefficient is -0.08150855508735176 and the corresponding robust standard error is 0.019586032782704106


Next, we run an ols regression of $Y$ on $(D,W)$ to control for the effect of covariates summarized in $W$:

\begin{align}
\log(Y) &=\beta_1 D  + \beta_2' W + \epsilon.
\end{align}

Here, we are considering the flexible model from the previous lab. Hence, $W$ controls for experience, education, region, and occupation and industry indicators plus transformations and two-way interactions. 
It's important to notice that we don't include the education controls since, as seen before, the values related to "shs","hsg","ad" are all $0$.

Let us run the ols regression with controls.

## Ols regression with controls

In [83]:
flex = @formula(lwage ~ sex + (exp1+exp2+exp3+exp4) * (clg+occ2+ind2+mw+so+we))
control_model = lm(flex , data)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

lwage ~ 1 + sex + exp1 + exp2 + exp3 + exp4 + clg + occ2 + ind2 + mw + so + we + exp1 & clg + exp1 & occ2 + exp1 & ind2 + exp1 & mw + exp1 & so + exp1 & we + exp2 & clg + exp2 & occ2 + exp2 & ind2 + exp2 & mw + exp2 & so + exp2 & we + exp3 & clg + exp3 & occ2 + exp3 & ind2 + exp3 & mw + exp3 & so + exp3 & we + exp4 & clg + exp4 & occ2 + exp4 & ind2 + exp4 & mw + exp4 & so + exp4 & we

Coefficients:
─────────────────────────────────────────────────────────────────────────────────────────
                        Coef.   Std. Error       t  Pr(>|t|)     Lower 95%      Upper 95%
─────────────────────────────────────────────────────────────────────────────────────────
(Intercept)       3.43576        0.520879     6.60    <1e-10    2.41442       4.45709
sex              -0.0530623      0.0193532   -2.74    0.0061   -

In [84]:
control_est = GLM.coef(control_model)[2]
control_se = GLM.coeftable(control_model).cols[2][2]
control_se1 = stderror( HC0(), control_model)[2]
println("Coefficient for OLS with controls " , control_est)

Coefficient for OLS with controls -0.05306234035918037


The estimated regression coefficient $\beta_1\approx-0.053$ measures how our linear prediction of wage changes if we set the gender variable $D$ from 0 to 1, holding the controls $W$ fixed.
We can call this the *predictive effect* (PE), as it measures the impact of a variable on the prediction we make. Overall, we see that the unconditional wage gap of size $8.1$\% for women decreases to about $5.3$\% after controlling for worker characteristics of the subset of college-educated workers.


Next, we are using the Frisch-Waugh-Lovell theorem from the lecture partialling-out the linear effect of the controls via ols.

## Partialling-Out using ols

In [85]:
# models
# model for Y
flex_y = @formula(lwage ~ (exp1+exp2+exp3+exp4) * (clg+occ2+ind2+mw+so+we))
flex_d = @formula(sex ~ (exp1+exp2+exp3+exp4) * (clg+occ2+ind2+mw+so+we))

# partialling-out the linear effect of W from Y
t_Y = residuals(lm(flex_y, data))

# partialling-out the linear effect of W from D
t_D = residuals(lm(flex_d, data))

data_res = DataFrame(t_Y = t_Y, t_D = t_D )
# regression of Y on D after partialling-out the effect of W

partial_fit = lm(@formula(t_Y ~ t_D), data_res)

partial_est = GLM.coef(partial_fit)[2]

# standard error
partial_se = GLM.coeftable(partial_fit).cols[2][2]

partial_se1 = stderror( HC0(), partial_fit)[2]

#condifence interval
GLM.confint(partial_fit)[2,:]

2-element Vector{Float64}:
 -0.08957069899693053
 -0.016553981718577013